## Brain Tumor Classification


### Clone the Github Repo to access the Dataset


### Import necessary Libraries 


In [ ]:
import torch, torchvision, os, random
## from tqdm import tqdm
## import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
## import seaborn as sns
## import cv2
## from IPython.display import Image
## import imutils

## from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


## import tensorflow.keras as keras
## from tensorflow.keras.models import Sequential

## import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchsummary import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau
## from torchvision.transforms import functional as F
## from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, array_to_img, img_to_array
## from tensorflow.keras.applications import EfficientNetB1
## from tensorflow.keras.models import Model
## from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, GlobalAveragePooling2D
## from tensorflow.keras.optimizers import Adam
## from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
## from kcn import ConvNeXtKAN

try:
    model_name
except NameError:
    model_name = 'ConvNeXtKAN'


if torch.cuda.is_available():
    print("Cuda is here!")
else:
    print("Cuda is not here :( ")


### Data Visualization

In [ ]:
files_path_dict = {}

# Hope you don't be imprisoned by legacy Python code :)
from pathlib import Path

# `cwd`: current directory is straightforward
cwd = Path.cwd()

train_dir = str(cwd) + "/data/Training"
test_dir = str(cwd) + "/data/Testing"

data_training = {
    'glioma_tumor': [],
    'meningioma_tumor': [],
    'no_tumor': [],
    'pituitary_tumor': []
}
for i in data_training.keys():
    for j in os.listdir(train_dir + "/" + i):
        data_training[i].append(f"{train_dir}/{i}/{j}")

data_testing = {
    'glioma_tumor': [],
    'meningioma_tumor': [],
    'no_tumor': [],
    'pituitary_tumor': []
}
for i in data_testing.keys():
    for j in os.listdir(test_dir + "/" + i):
        data_testing[i].append(f"{test_dir}/{i}/{j}")

len_train = {}
for i in data_training.keys():
    len_train[i] = len(data_training[i])
len_testing = {}
for i in data_testing.keys():
    len_testing[i] = len(data_testing[i])

print({"training": len_train, "testing": len_testing})


In [ ]:
plt.figure(figsize=(17, 17))
index = 0
for c in data_training.keys():
    random.shuffle(data_training[c])
    path_list = data_training[c][:5]

    for i in range(1, 5):
        index += 1
        plt.subplot(4, 4, index)
        plt.imshow(load_img(path_list[i]))
        plt.title(c)


### Data Augmentation

In [ ]:
# Define transformations
train_transform = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=0, translate=(0, 0.2)),
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

valid_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# Split the training data into training and validation sets
num_train = len(train_dataset)
split = int(0.8 * num_train)  # 80% training, 20% validation
train_size = split
valid_size = num_train - split

train_dataset, valid_dataset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
"train: ", len(train_dataset), "Valid: ", len(valid_dataset), "test: ", len(test_dataset)

### Compile Model

In [ ]:
# Instancujte model
model = torchvision.models.efficientnet_b1(progress=True)

if torch.cuda.is_available():
    model.cuda()


# Vytiskněte souhrn modelu
def print_model_summary(model, input_size=(3, 128, 128)):
    summary(model, input_size=input_size)


print_model_summary(model)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.CrossEntropyLoss()


### Model Training and Model Evaluation

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_score = None
        self.early_stop_counter = 0
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        if self.best_score is None:
            self.best_score = val_loss
            self.best_model_wts = model.state_dict()
        elif val_loss < self.best_score:
            self.best_score = val_loss
            self.best_model_wts = model.state_dict()
            self.early_stop_counter = 0
        else:
            self.early_stop_counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.early_stop_counter} out of {self.patience}')
            if self.early_stop_counter >= self.patience:
                if self.verbose:
                    print('Early stopping')
                model.load_state_dict(self.best_model_wts)
                return True
        return False

# Define the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.3, patience=2, verbose=True)

# Initialize early stopping
early_stopping = EarlyStopping(patience=5, verbose=True)



In [5]:
%%time

accuracy_training = []
accuracy_testing = []
loss_validation = []
loss_training = []

try:
    num_epoch
except NameError:
    num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    epoch_acc = 0

    # Training loop with progress bar
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for inputs, labels in train_loader:

            if torch.cuda.is_available():
                inputs = inputs.to(torch.device('cuda'))
                labels = labels.to(torch.device('cuda'))

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar
            pbar.set_postfix(loss=running_loss / (total + 1e-8), accuracy=correct / total)
            pbar.update()

            # logging 
            accuracy_training.append(correct / total)
            loss_training.append(running_loss / (total + 1e-8))

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total
    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}')

    accuracy_training.append(epoch_acc)
    loss_training.append(epoch_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    # Validation loop with progress bar
    with tqdm(total=len(valid_loader), desc='Validation', unit='batch') as pbar:
        with torch.no_grad():
            for inputs, labels in valid_loader:

                if torch.cuda.is_available():
                    inputs = inputs.to(torch.device('cuda'))
                    labels = labels.to(torch.device('cuda'))

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                # Update progress bar
                pbar.set_postfix(val_loss=val_loss / (val_total + 1e-8), val_accuracy=val_correct / val_total)
                pbar.update()

    val_loss = val_loss / len(valid_dataset)
    val_acc = val_correct / val_total
    print(f'Validation Loss: {val_loss:.4f} - Accuracy: {val_acc:.4f}')

    loss_validation.append(val_loss)
    accuracy_testing.append(val_acc)

    # Step the scheduler
    scheduler.step(val_acc)

    # Check early stopping
    if early_stopping(val_loss, model):
        num_epochs = epoch
        break

2024-08-02 15:14:25.195206: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 15:14:25.321342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 15:14:25.340893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 15:14:25.539614: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-02 15:14:27.842936: W tensorflow/compiler/tf2

NameError: name 'model' is not defined

In [1]:
from common import save_checkpoint

save_checkpoint(model, optimizer, model_name + "_checkpoint.pth", num_epochs)

NameError: name 'accuracy_testing' is not defined

### Model Evaluation

In [ ]:
from common import load_checkpoint

model, optimizer, num_epochs = load_checkpoint(model, optimizer, model_name + "_checkpoint.pth", )


In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

epochs = np.arange(0, num_epochs, num_epochs / len(accuracy_training))

ax[0].plot(epochs, accuracy_training, 'g', label='Training Accuracy')

epochs = np.arange(0, num_epochs, num_epochs / len(accuracy_testing))

ax[0].plot(epochs, accuracy_testing, 'y-', label='Validation Accuracy')
ax[0].set_title('Model Training & Validation Accuracy')
ax[0].legend(loc='lower right')
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")
epochs = np.arange(0, num_epochs, num_epochs / len(loss_training))

ax[1].plot(epochs, loss_training, 'g', label='Training Loss')

epochs = np.arange(0, num_epochs, num_epochs / len(loss_validation))

ax[1].plot(epochs, loss_validation, 'y-', label='Validation Loss')
ax[1].set_title('Model Training & Validation & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")

plt.savefig(model_name + ".svg")


In [ ]:
from common import update_json_with_key

update_json_with_key('data.json', model_name, num_epochs, accuracy_training, accuracy_testing, loss_training,
                     loss_validation)
